In [162]:
import LiveAuctionProcessor
import importlib
import time
import psycopg2 as pg2
import pandas as pd
from datetime import datetime, timedelta

In [233]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='')

print ("Reading Dataset")
df = pd.read_sql ("""Select *  from auction_full where auctiontime >= '2020-01-01' LIMIT 1""", conn)


Connecting to SQL
Reading Dataset


In [111]:
auction_id = '620712999'
importlib.reload(LiveAuctionProcessor)
auction_dict = {"auction_id":'620712999'}
lap = LiveAuctionProcessor.LiveAuctionProcessor(auction_dict)


In [112]:
from IPython.display import clear_output

while (True):
    clear_output()
    bh = lap.update()
    for b in bh[-5:]:
        print (b)
    time.sleep(1)

{'bid': 88, 'usename': 'mantinori', 'is_bidomatic': True}
{'bid': 89, 'usename': 'jsmthjm', 'is_bidomatic': True}
{'bid': 90, 'usename': 'mantinori', 'is_bidomatic': True}
{'bid': 91, 'usename': 'jsmthjm', 'is_bidomatic': True}
{'bid': 92, 'usename': 'mantinori', 'is_bidomatic': True}


KeyboardInterrupt: 

In [196]:
bh = [{'bid': 1, 'username': 'mantinori', 'is_bidomatic': False},
 {'bid': 2, 'username': 'twylla67', 'is_bidomatic': False},
 {'bid': 3, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 4, 'username': 'Boomer2107', 'is_bidomatic': False},
 {'bid': 5, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 6, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 7, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 8, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 9, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 10, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 11, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 12, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 13, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 14, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 15, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 16, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 17, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 18, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 19, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 20, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 21, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 22, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 23, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 24, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 25, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 26, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 27, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 28, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 29, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 30, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 31, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 32, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 33, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 34, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 35, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 36, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 37, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 38, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 39, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 40, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 41, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 42, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 43, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 44, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 45, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 46, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 47, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 48, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 49, 'username': 'jsmthjm', 'is_bidomatic': False},
 {'bid': 50, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 51, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 52, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 53, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 54, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 55, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 56, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 57, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 58, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 59, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 60, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 61, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 62, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 63, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 64, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 65, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 66, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 67, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 68, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 69, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 70, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 71, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 72, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 73, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 74, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 75, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 76, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 77, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 78, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 79, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 80, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 81, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 82, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 83, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 84, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 85, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 86, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 87, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 88, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 89, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 90, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 91, 'username': 'jsmthjm', 'is_bidomatic': True},
 {'bid': 92, 'username': 'mantinori', 'is_bidomatic': True}]

In [198]:
bh

[{'bid': 1, 'username': 'mantinori', 'is_bidomatic': False},
 {'bid': 2, 'username': 'twylla67', 'is_bidomatic': False},
 {'bid': 3, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 4, 'username': 'Boomer2107', 'is_bidomatic': False},
 {'bid': 5, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 6, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 7, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 8, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 9, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 10, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 11, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 12, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 13, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 14, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 15, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 16, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 17, 'username':

In [234]:
bh_df = pd.DataFrame.from_dict(bh)

In [235]:
df.columns

Index(['auctionid', 'is_winner', 'cardtype', 'cashvalue', 'cardvalue', 'fee',
       'bidvalue', 'limited_allowed', 'is_locked', 'auctiontime', 'bid',
       'is_bidomatic', 'bids_so_far', 'username', 'prevusers', 'giveup',
       'eventual_bids', 'eventual_win_price', 'debut', 'bom_streak',
       'bom_bids_so_far', 'perc_to_bin', 'username0', 'distance0',
       'is_bidomatic0', 'bids_so_far0', 'bom_bids_so_far0', 'bom_streak0',
       'perc_to_bin0', 'prev_is_new_user0', 'prev_auction_count0',
       'prev_overbid0', 'prev_giveup_one0', 'prev_give_before_six0',
       'prev_wins0', 'prev_bids0', 'prev_bom_bids0', 'username1', 'distance1',
       'is_bidomatic1', 'bids_so_far1', 'bom_bids_so_far1', 'bom_streak1',
       'perc_to_bin1', 'prev_is_new_user1', 'prev_auction_count1',
       'prev_overbid1', 'prev_giveup_one1', 'prev_give_before_six1',
       'prev_wins1', 'prev_bids1', 'prev_bom_bids1', 'username2', 'distance2',
       'is_bidomatic2', 'bids_so_far2', 'bom_bids_so_far2', 

In [273]:
auction_dict["is_winner"] = 0
auctiontime = datetime.now()+ timedelta(hours=1)
bid = len(bh)+1
is_bidomatic = True
bids_so_far = '0'
my_username = 'AAAAAAAHH'
user_dict = {}
cur_opponent = 0
found_users = {}
for r in bh[-1::-1]:
    username = r['username']
    cur_bid = r['bid']
    is_bidomatic = r['is_bidomatic']
    if username == my_username:
        break
    if (username not in found_users.keys()):
        user_dict[cur_opponent]={"username":username, "bids_so_far": 0, "bom_bids_so_far": 0, "distance": bid-cur_bid, "is_bidomatic": is_bidomatic, "bom_streak": 0,  "bom_streak_on": True}
        found_users[username] = cur_opponent
        cur_opponent += 1
    my_row = user_dict[found_users[username]] 
    my_row ["bids_so_far"] += 1
    if (is_bidomatic):
        my_row["bom_bids_so_far"]+=1
        if my_row["bom_streak_on"]:
            my_row["bom_streak"] += 1
    
    else:
        my_row["bom_streak_on"] = False
prevusers = len(user_dict)
giveup = None
eventual_bids = None
debut = None
bom_streak = None
bom_bids_so_far = None
perc_to_bin = None



In [274]:
user_dict

{0: {'username': 'mantinori',
  'bids_so_far': 45,
  'bom_bids_so_far': 44,
  'distance': 1,
  'is_bidomatic': True,
  'bom_streak': 44,
  'bom_streak_on': False},
 1: {'username': 'jsmthjm',
  'bids_so_far': 20,
  'bom_bids_so_far': 19,
  'distance': 2,
  'is_bidomatic': True,
  'bom_streak': 19,
  'bom_streak_on': False},
 2: {'username': 'Boomer2107',
  'bids_so_far': 26,
  'bom_bids_so_far': 25,
  'distance': 38,
  'is_bidomatic': True,
  'bom_streak': 25,
  'bom_streak_on': False},
 3: {'username': 'twylla67',
  'bids_so_far': 1,
  'bom_bids_so_far': 0,
  'distance': 91,
  'is_bidomatic': False,
  'bom_streak': 0,
  'bom_streak_on': False}}

In [256]:
bh


[{'bid': 1, 'username': 'mantinori', 'is_bidomatic': False},
 {'bid': 2, 'username': 'twylla67', 'is_bidomatic': False},
 {'bid': 3, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 4, 'username': 'Boomer2107', 'is_bidomatic': False},
 {'bid': 5, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 6, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 7, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 8, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 9, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 10, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 11, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 12, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 13, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 14, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 15, 'username': 'mantinori', 'is_bidomatic': True},
 {'bid': 16, 'username': 'Boomer2107', 'is_bidomatic': True},
 {'bid': 17, 'username':

In [252]:
bh_df.groupby('username').sum()

,bid,is_bidomatic
username,,
Boomer2107,757,25.0
jsmthjm,1435,19.0
mantinori,2084,44.0
twylla67,2,0.0


In [251]:
df.username[bh_df.is_bidomatic

KeyError: 'twylla67'